# 데이터 전처리

## 데이터 전처리의 주요 작업
- 데이터 정제(cleaning)
    - 결측치(missing values)의 파악과 채워 넣기
    - 이상치(outliers)를 식별하여 필요한 경우 제거하기
    - 불일치(discrepancies)를 해소하기
        - 잘못된 대답 등
- 데이터 통합(integration)
    - 다수의 데이터베이스와 데이터 큐브, 혹은 파일들의 통합
    - 개체 식별의 문제 : 스키마 통합시 메타데이터를 사용하여 객체 매칭 작업
        - ex) A파일 x변수명 : 'sex' / B파일 x 변수명 : 'gender'
- 데이터 변환(transformation)
    - 연속형 변수 : 데이터가 일정 구간에 속하도록 정규화(normalization)
    - 범주형 변수 : 문자열은 수치로 코딩, 원-핫-인코딩(one-hot-encoding)
    - 훈련 데이터와 테스트 데이터 모두에 동일한 변환을 적용해야 함
- 데이터 축소(reduction)
    - 거의 동일한 분석 결과를 주는 범위 내에서 데이터 양을 추곳
    - 차원 축소(dimension reduction)
    - 데이터 이산화(discretization)

# 범주형 데이터 다루기

## 범주형 변수의 유형
- 학습 알고리즘의 입력으로 사용하기 위해서는 문자열을 정수로 변환하여야 함
- 범주형 : 순서형(ordinal) vs 명목형(nomial)
- sklearn 형태
    - 추정기(estimator) : LogisticRegression, SVC, ......
        - fit -> predict -> score 메서드
    - 변환기(transtormer) : LabelEncoder, StandardScaler, ...
        - fit -> fransform 메서드
        - 아니면 fit_tramsform 메서드

In [1]:
import pandas as pd
import numpy as np

df = pd.DataFrame([
    ['green', 'L', 10.1, 'yes'],
    ['green', 'M', 13.5, 'no'],
    ['red', 'XL', 15.3, 'yes'],
    ['blue', 'S', 12.7, 'yes']
])
# 명목, 순서, 수치, 명목
df.columns = ['color', 'size', 'price', 'label']
df

,color,size,price,label
0,green,L,10.1,yes
1,green,M,13.5,no
2,red,XL,15.3,yes
3,blue,S,12.7,yes


In [2]:
X = df.drop('label', axis = 1)
X

,color,size,price
0,green,L,10.1
1,green,M,13.5
2,red,XL,15.3
3,blue,S,12.7


In [20]:
y = df.label
y

0    yes
1     no
2    yes
3    yes
Name: label, dtype: object

#### 타겟이 명목형인 경우 인코딩

In [21]:
# 타겟이 명목형, 인코딩(1)
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y)
label_encoder.classes_

array(['no', 'yes'], dtype=object)

In [23]:
y = label_encoder.transform(y)  # 순서대로 0, 1 부여
y

array([1, 0, 1, 1])

In [28]:
# 정수 레이블을 원래 문자열로 돌려놓기
y = label_encoder.inverse_transform(y)
y

array(['yes', 'no', 'yes', 'yes'], dtype=object)

In [29]:
# 타겟이 명목형, 인코딩(2)
y = label_encoder.fit_transform(y)
y

array([1, 0, 1, 1])

#### 순서형 특성 매핑 : size 변환
1. 딕셔너리를 이용한 mapping
2. np.where를 이용
3. LabelEncoder

In [30]:
X

,color,size,price
0,green,L,10.1
1,green,M,13.5
2,red,XL,15.3
3,blue,S,12.7


In [53]:
# 1
d = {'S' : 0, 'M' : 1, 'L' : 2, 'XL' : 3}
X['size_transformed1'] = X['size'].map(d)
X

,color,size,price,size_transformed1
0,green,L,10.1,2
1,green,M,13.5,1
2,red,XL,15.3,3
3,blue,S,12.7,0


In [54]:
# 2
X = X.drop('size_transformed1', axis = 1) 

X['size_transformed2'] = np.where(X['size'] == 'S', 0, 
                                  np.where(X['size'] == 'M', 1, 
                                           np.where(X['size'] == 'L', 2, 
                                                    np.where(X['size'] == 'XL', 3, 'np.nan'))))
X

,color,size,price,size_transformed2
0,green,L,10.1,2
1,green,M,13.5,1
2,red,XL,15.3,3
3,blue,S,12.7,0


In [55]:
# 3
X = X.drop('size_transformed2', axis = 1) 

from sklearn.preprocessing import LabelEncoder
X_encoder = LabelEncoder()
X['size_transformed3'] = X_encoder.fit_transform(X['size'])
X

,color,size,price,size_transformed3
0,green,L,10.1,0
1,green,M,13.5,1
2,red,XL,15.3,3
3,blue,S,12.7,2


#### 명목형 특성에 대한 원-핫-인코딩(one-hot-encoding)
- 명목형 범주형 데이터는 정수를 매핑하면 안 되고, 고유한 값(범주)마다 가변수(dummy: 0과 1만 가지는 변수) 특성을 만들어야 함
- k개의 범주를 갖는 변수는 k개의 가변수(dummy)들이 필요
- pd.get_dummies

In [66]:
X = X.drop('size', axis = 1)
X

,color,price,size_transformed3
0,green,10.1,0
1,green,13.5,1
2,red,15.3,3
3,blue,12.7,2


In [68]:
X_encorded = pd.get_dummies(X)
X_encorded

,price,size_transformed3,color_blue,color_green,color_red
0,10.1,0,0,1,0
1,13.5,1,0,1,0
2,15.3,3,0,0,1
3,12.7,2,1,0,0


# 와인 데이터
- 178개 와인 샘플에 대한 13개의 화학성분(특성)과 와인 포도 품종(클래스 레이블)

In [70]:
path = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data'
wine = pd.read_csv(path, header=None)
wine.columns = ['Class label', 'Alcohol', 'Malic acid', 'Ash',
 'Alcalinity of ash', 'Magnesium',
 'Total phenols', 'Flavanoids', 'Nonflavanoid phenols',
 'Proanthocyanins', 'Color intensit', 'Hue',
 'OD280/OD315 of diluted wines', 'Proline']
wine.head(2)

,Class label,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensit,Hue,OD280/OD315 of diluted wines,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050


In [71]:
X = wine.iloc[:, 1:].values  # to_numpy()
y = wine.iloc[:, 0].values  # to_numpy()

- 데이터 분할

In [73]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, stratify=y)

## 데이터 변환
- 데이터 특성 스케일 조정하기
    - 결정트리를 제외한 머신러닝 알고리즘은 특성의 스케일을 조정할 때 더 나은 성능을 보임
    - **훈련 데이터에 대한 변환 공식과 동일한 공식**으로 테스트 데이터도 변환해주어야 함

#### 표준정규분포로 표준화
- sklearn.preprocessing.StandardScaler
- $z = \frac{x-평균}{표준편차}$
- 대략 -3 ~ 3 사이로 변환

In [76]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)  # 훈련 데이터에 대한 변환 공식과 동일한 공식 사용

#### 0~1 사이로 정규화
- sklearn.preprocessing.MinMaxScaler
- $z = \frac{(x-min)}{(max-min)}$

In [75]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)  # 훈련 데이터에 대한 변환 공식과 동일한 공식 사용

## 차원 축소


- 차원 축소(dimensionality reduction) = 특성 추출(feature extraction)
    - 특성들의 조합으로 새로운 특성을 생성
    - 원래의 특성 공간 대신 새로운 저차원  특성 공간으로 데이터를 투영
    - 대표적 차원 축소 알고리즘 : PCA, LDA, 커널 PCA

- 주성분 분석(PCA : Princepal Component Anlysis)
    - 특성들이 통계적으로 상관관계가 없도록 데이터를 회전시키는 기술
    - 분산이 가장 큰(정보가 가장 큰) 방향을 첫 번째 성분으로, 첫 번째 성분과 직각인 방향 중에서 분산이 가장 큰 방향을 두 번째 성분으로 하는 과정을 반복
    - <img src = 'https://miro.medium.com/max/1200/0*l5-imeQ9RfmaVWtL'/>  
    
    - 특성행렬의 공분산행렬에 대한 고유치(eigenvalues)와 고유벡터(eigenvector)를 사용
        - 투영 행렬(projection matrix) $W$ : 고유치가 큰 상위 k개의 고유벡터로 이루어짐
        - $X' = X W$ : 투영된 데이터
    - sklearn.decomposition.PCA

In [80]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)  # 주성분의 개수를 2개로 설정
pca.fit(X_train_std)
pca.components_, pca.explained_variance_ratio_ 

(array([[ 0.10796674, -0.23528314, -0.02192287, -0.23758476,  0.09019433,
          0.40354786,  0.42624578, -0.30309479,  0.31928845, -0.13425956,
          0.30189274,  0.38711239,  0.26562025],
        [-0.48311991, -0.19600038, -0.30092282, -0.01413707, -0.33933788,
         -0.09541662, -0.0220313 ,  0.01217124, -0.07714306, -0.50391344,
          0.27172291,  0.1503662 , -0.39523843]]),
 array([0.35049367, 0.19835018]))

components_ : 투영 행렬 $W$  
explained_variance_ratio_ : 각 주성분으로 설명되는 분산의 비율(요소들의 합 = 설명되는 비율)
    - 분산의 비율을 보면서 주성분 개수를 조절

In [81]:
X_train_pca = pca.transform(X_train_std)
X_test_pca = pca.transform(X_test_std)

- 주성분에 대한 로지스틱 회귀(두 개의 주성분)

In [83]:
from sklearn.linear_model import LogisticRegression
logistic = LogisticRegression()
logistic.fit(X_train_pca, y_train)
logistic.score(X_test_pca,  y_test)

0.9259259259259259